# Liver Model Construction: Glycolysis Gluconeogenesis & Lactate Fermentation

## Setup workflow

### Import packages

In [1]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sy
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass import MassMetabolite, MassModel, MassReaction
from mass.io.json import save_json_model as save_json_mass_model

print("MASSpy version: {0}".format(mass.__version__))

MASSpy version: 0.1.1


## Set paths and constants

### Directory paths

## Change this before upload!!

model_dir = os.path.abspath("../models")
maps_dir = os.path.abspath("../maps")

In [2]:
# **changed the path depending on the file on where the files are located**
model_dir = os.path.abspath("/Users/RiyaMathur/MASSliver/models")
maps_dir = os.path.abspath("/Users/RiyaMathur/MASSliver/models/maps")
data_dir = os.path.abspath("/Users/RiyaMathur/MASSliver/data/interim")

# Create MASS model from COBRA model

### Load COBRA model

In [3]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
recon=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
core_model

Name,None
Memory address,0x07fcfec41a490
Number of metabolites,178
Number of reactions,187
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


## Create MASS model

In [4]:
glycogenolysis = MassModel("Glycogenolysis", array_type='DataFrame', dtype=np.int64)

### Define reactions

In [5]:
# glycolysis_map = escher.Builder(model=core_model,  map_json="Glycolysis, gluconeogenesis lactate fermentation 2.json",
#                                 highlight_missing=True) 
# glycolysis_map

In [6]:
reaction_list = ['HEX1',
                'PGI',
                'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L',
              # 'PYRt2m', This is being taken from Reconn (below) since CORE has it in inner mitochodria (h_i)
              #  'SUCR', This is a sucrose metabolism reaction... 
                'G6PDH2r',
                'PGMT',
                'G6Pter',
                'GLCter',
                'TKT2',
                'PGCD', 
                '3SPYRSP', 
                'ALATA_L', 
                '2AMACHYD', 
               # 'GLCt1', This is a surcose metabolism reaction...
                'PEPCK_re',
                'MDH',
                'MALtm', 
                'EX_glc__D_c',
                'EX_pyr_c',
                'EX_lac__L_c',
                'G6Pter',
                'G6PPer',
                'GLCter']



new = ['EX_glc__D_e', 'GLCt1', 'PYRt2m', 'H2Oter', 'PIter', 'H2Ot', 'PIt', 'EX_pi_e', 
       'EX_h2o_e', 'EX_h_e', 'Ht',  'L_LACt2r', #lactose transport between lactate in cytosol and extracellular 
     'PYRt2'] #pyruvate transport between pyruvate in cytosol and extracellular] #from RECONN


new_reaction_list = []
for item in reaction_list: 
    item = core_model.reactions.get_by_id(item)
    new_reaction_list.append(item) 

new_reaction_list
for i in new: 
    i = recon.reactions.get_by_id(i)
    new_reaction_list.append(i) 

# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])
for rid in new:
    reaction = recon.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])  
new_reaction_list

## This section is adding the sinks that are necessary 

#pyr_c sink ** Just keeping for now... will not include in FBA is if not working
metabolite = recon.metabolites.get_by_id('pyr_c')

SK_pyr_c = glycogenolysis.add_boundary(
    metabolite= 'pyr_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_pyr_c)]) 
new_reaction_list.append(SK_pyr_c)

#pyr_m sink...will not include in FBA is if not working
metabolite = recon.metabolites.get_by_id('pyr_m')

SK_pyr_m = glycogenolysis.add_boundary(
    metabolite= 'pyr_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_pyr_m)]) 
new_reaction_list.append(SK_pyr_m)

#hco3_m sink
metabolite = recon.metabolites.get_by_id('hco3_m')

SK_hco3_m = glycogenolysis.add_boundary(
    metabolite= 'hco3_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_hco3_m)]) 
new_reaction_list.append(SK_hco3_m)

#co2_m sink
metabolite = recon.metabolites.get_by_id('co2_m')

SK_co2_m = glycogenolysis.add_boundary(
    metabolite= 'co2_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_co2_m)]) 
new_reaction_list.append(SK_co2_m)

#gtp_m sink
metabolite = recon.metabolites.get_by_id('gtp_m')

SK_gtp_m = glycogenolysis.add_boundary(
    metabolite= 'gtp_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_gtp_m)]) 
new_reaction_list.append(SK_gtp_m)

#gdp_m sink
metabolite = recon.metabolites.get_by_id('gdp_m')

SK_gdp_m = glycogenolysis.add_boundary(
    metabolite= 'gdp_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_gdp_m)]) 
new_reaction_list.append(SK_gdp_m)

#atp_c sink
metabolite = recon.metabolites.get_by_id('atp_c')

SK_atp_c = glycogenolysis.add_boundary(
    metabolite= 'atp_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_atp_c)]) 
new_reaction_list.append(SK_atp_c)

#atp_m sink
metabolite = recon.metabolites.get_by_id('atp_m')

SK_atp_m = glycogenolysis.add_boundary(
    metabolite= 'atp_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_atp_m)]) 
new_reaction_list.append(SK_atp_m)

#adp_c sink
metabolite = recon.metabolites.get_by_id('adp_c')

SK_adp_c = glycogenolysis.add_boundary(
    metabolite= 'adp_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_adp_c)]) 
new_reaction_list.append(SK_adp_c)

#pi_m sink
metabolite = recon.metabolites.get_by_id('pi_m')

SK_pi_m = glycogenolysis.add_boundary(
    metabolite= 'pi_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_pi_m)]) 
new_reaction_list.append(SK_pi_m)

#h_m sink
metabolite = recon.metabolites.get_by_id('h_m')

SK_h_m = glycogenolysis.add_boundary(
    metabolite= 'h_m', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_h_m)]) 
new_reaction_list.append(SK_h_m)

#nadh_c sink
metabolite = recon.metabolites.get_by_id('nadh_c')

SK_nadh_c = glycogenolysis.add_boundary(
    metabolite= 'nadh_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_nadh_c)]) 
new_reaction_list.append(SK_nadh_c)

#nad_c sink
metabolite = recon.metabolites.get_by_id('nad_c')

SK_nad_c = glycogenolysis.add_boundary(
    metabolite= 'nad_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_nad_c)]) 
new_reaction_list.append(SK_nad_c)

## This section is adding the exchanges that are necessary
#h2o, pie, he, pyre, lace

#pyr_e exchange
metabolite = recon.metabolites.get_by_id('pyr_e')

EX_pyr_e = glycogenolysis.add_boundary(
    metabolite= 'pyr_e', boundary_type="exchange", subsystem="Pseudoreaction")

glycogenolysis.add_reactions([MassReaction(EX_pyr_e)]) 
new_reaction_list.append(EX_pyr_e)

#lac__L_e exchange
metabolite = recon.metabolites.get_by_id('lac__L_e')

EX_lac__L_e = glycogenolysis.add_boundary(
    metabolite= 'lac__L_e', boundary_type="exchange", subsystem="Pseudoreaction")

glycogenolysis.add_reactions([MassReaction(EX_lac__L_e)]) 
new_reaction_list.append(EX_lac__L_e)

print(new_reaction_list)

Ignoring reaction 'G6Pter' since it already exists.
Ignoring reaction 'GLCter' since it already exists.
Ignoring reaction 'SK_pyr_c' since it already exists.
Ignoring reaction 'SK_pyr_m' since it already exists.
Ignoring reaction 'SK_hco3_m' since it already exists.
Ignoring reaction 'SK_co2_m' since it already exists.
Ignoring reaction 'SK_gtp_m' since it already exists.
Ignoring reaction 'SK_gdp_m' since it already exists.
Ignoring reaction 'SK_atp_c' since it already exists.
Ignoring reaction 'SK_atp_m' since it already exists.
Ignoring reaction 'SK_adp_c' since it already exists.
Ignoring reaction 'SK_pi_m' since it already exists.
Ignoring reaction 'SK_h_m' since it already exists.
Ignoring reaction 'SK_nadh_c' since it already exists.
Ignoring reaction 'SK_nad_c' since it already exists.
Ignoring reaction 'EX_pyr_e' since it already exists.
Ignoring reaction 'EX_lac__L_e' since it already exists.


[<Reaction HEX1 at 0x7fcff521a190>, <Reaction PGI at 0x7fcff521a970>, <Reaction FBP at 0x7fcff5223e50>, <Reaction PFK at 0x7fcff521ae50>, <Reaction FBA at 0x7fcff521a220>, <Reaction TPI at 0x7fcff521a1c0>, <Reaction GAPD at 0x7fcff521adf0>, <Reaction PGK at 0x7fcff521ad30>, <Reaction PGM at 0x7fcff51effa0>, <Reaction ENO at 0x7fcff51ef190>, <Reaction PEPtm at 0x7fcff52ecf10>, <Reaction PEPCKm at 0x7fcff52233d0>, <Reaction PYK at 0x7fcff5331820>, <Reaction PCm at 0x7fcff520af70>, <Reaction LDH_L at 0x7fcff5203bb0>, <Reaction G6PDH2r at 0x7fcff52038e0>, <Reaction PGMT at 0x7fcff52288e0>, <Reaction G6Pter at 0x7fcff5223370>, <Reaction GLCter at 0x7fcff5223a00>, <Reaction TKT2 at 0x7fcff520a5b0>, <Reaction PGCD at 0x7fcff535a3d0>, <Reaction 3SPYRSP at 0x7fcff535a070>, <Reaction ALATA_L at 0x7fcff5354e20>, <Reaction 2AMACHYD at 0x7fcff532ffa0>, <Reaction PEPCK_re at 0x7fcff52289a0>, <Reaction MDH at 0x7fcff52284c0>, <Reaction MALtm at 0x7fcff5228760>, <Reaction EX_glc__D_c at 0x7fcff5370fa0

### View extracted network

In [7]:
escher_builder = escher.Builder(
    model=glycogenolysis,
    map_json=os.path.join(
        model_dir, ".".join((
            glycogenolysis.id, "map", "json"))
    ),
    highlight_missing=True)

escher_builder

ValueError: Could not load map_json. Not valid json, url, or filepath

## Parameterize MASS model

# Export MASS model

In [8]:
save_json_mass_model(
    mass_model=glycogenolysis,
    filename=os.path.join(model_dir, glycogenolysis.id + ".json"))